In [ ]:
import numpy as np

# For reproducibility
np.random.seed(0)

# Hidden states (phonemes)
states = ['/s/', '/p/', '/ie:/', '/tS/']

# Observations (acoustic properties)
observations = ['Energy', 'Pitch', 'Duration']

# Initial probabilities: start always at /s/
pi = {
    '/s/': 1.0,
    '/p/': 0.0,
    '/ie:/': 0.0,
    '/tS/': 0.0
}

# Transition probabilities (deterministic sequence for the word "speech")
# /s/ → /p/ → /ie:/ → /tS/
A = {
    '/s/':  {'/p/': 1.0},
    '/p/':  {'/ie:/': 1.0},
    '/ie:/':{'/tS/': 1.0},
    '/tS/': {'/tS/': 1.0}   # absorb at last phoneme
}

# Emission probabilities (given in the question)
# Phoneme → Observation: Energy, Pitch, Duration
B = {
    '/s/':  {'Energy': 0.7, 'Pitch': 0.2, 'Duration': 0.1},
    '/p/':  {'Energy': 0.5, 'Pitch': 0.3, 'Duration': 0.2},
    '/ie:/':{'Energy': 0.3, 'Pitch': 0.5, 'Duration': 0.2},
    '/tS/': {'Energy': 0.4, 'Pitch': 0.4, 'Duration': 0.2}
}


In [ ]:
import pandas as pd

def display_hmm_parameters(states, observations, pi, A, B):
    # Initial probabilities
    print("=== Initial Probabilities (π) ===")
    pi_df = pd.DataFrame.from_dict(pi, orient='index', columns=['Probability'])
    display(pi_df)

    # Transition matrix
    print("\n=== Transition Probabilities (A) ===")
    # Ensure all states appear as columns, fill missing with 0
    A_full = {s_from: {s_to: A.get(s_from, {}).get(s_to, 0.0) for s_to in states}
              for s_from in states}
    A_df = pd.DataFrame.from_dict(A_full, orient='index', columns=states)
    display(A_df)

    # Emission matrix
    print("\n=== Emission Probabilities (B) ===")
    B_full = {state: {obs: B[state].get(obs, 0.0) for obs in observations}
              for state in states}
    B_df = pd.DataFrame.from_dict(B_full, orient='index', columns=observations)
    display(B_df)

# Call the function to display
display_hmm_parameters(states, observations, pi, A, B)


=== Initial Probabilities (π) ===


,Probability
/s/,1.0
/p/,0.0
/ie:/,0.0
/tS/,0.0



=== Transition Probabilities (A) ===


,/s/,/p/,/ie:/,/tS/
/s/,0.0,1.0,0.0,0.0
/p/,0.0,0.0,1.0,0.0
/ie:/,0.0,0.0,0.0,1.0
/tS/,0.0,0.0,0.0,1.0



=== Emission Probabilities (B) ===


,Energy,Pitch,Duration
/s/,0.7,0.2,0.1
/p/,0.5,0.3,0.2
/ie:/,0.3,0.5,0.2
/tS/,0.4,0.4,0.2


In [ ]:
def sample_from_distribution(prob_dict):
    """
    prob_dict: dict like {'Energy': 0.7, 'Pitch': 0.2, 'Duration': 0.1}
    Returns: one key sampled according to the given probabilities.
    """
    keys = list(prob_dict.keys())
    probs = np.array(list(prob_dict.values()))
    return np.random.choice(keys, p=probs)


In [ ]:
def generate_hmm_sequence(num_steps=4):
    phoneme_seq = []
    observation_seq = []

    # Start state from initial distribution (here always '/s/')
    current_state = sample_from_distribution(pi)

    for t in range(num_steps):
        phoneme_seq.append(current_state)

        # Sample observation given current phoneme
        obs = sample_from_distribution(B[current_state])
        observation_seq.append(obs)

        # Sample next state using transition probabilities
        # If current_state has no outgoing transitions, stay there
        if current_state in A and len(A[current_state]) > 0:
            next_state = sample_from_distribution(A[current_state])
        else:
            next_state = current_state

        current_state = next_state

    return phoneme_seq, observation_seq

# Generate one sequence for the word "speech"
phonemes, acoustic_obs = generate_hmm_sequence(num_steps=4)

print("Generated phoneme sequence:", phonemes)
print("Generated acoustic observations:", acoustic_obs)


Generated phoneme sequence: [np.str_('/s/'), np.str_('/p/'), np.str_('/ie:/'), np.str_('/tS/')]
Generated acoustic observations: [np.str_('Pitch'), np.str_('Pitch'), np.str_('Pitch'), np.str_('Duration')]
